In [34]:
import os
import numpy as np

from bisect import bisect_right, bisect_left

In [35]:
class Rectangle():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        
    def GetCentroid(self):
        return (self.x + self.width/2.0, self.y + self.height/2.0)

In [36]:
class UUIDRectangle(Rectangle):
    def __init__(self, x, y, width, height, uuid):
        super().__init__(x, y, width, height)
        self.assigned_to_set = False
        self.uuid = uuid

In [37]:
def CheckIntersectionOfRectangle(a, b):
    # Check if rectangles are either side of eachother
    if (a.x >= b.x + b.width or b.x >= a.x + a.width):
        return False;
    
    # Check if rectangles are above or below eachother
    if (a.y >= b.y + b.height or b.y >= a.y + a.height):
        return False;
    
    return True

In [38]:
def CheckCandidateIntersectsWithSetMembers(candidate, maximal_set):    
    for rectangle in maximal_set:
        if not CheckIntersectionOfRectangle(candidate, rectangle):
            return False
        
    return True

In [39]:
# Unit test x axis
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(0, 1, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == True

# Unit test x axis
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(0, 30, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == False

In [40]:
# Unit test y axis
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(1, 0, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == True

# Unit test y axis
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(30, 0, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == False

In [41]:
# Unit test same lower left corner point
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(0, 0, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == True

# Unit test lower left corner and upper right corner
rectangle_a = Rectangle(0, 0, 5, 5)
rectangle_b = Rectangle(5, 5, 5, 5)
assert CheckIntersectionOfRectangle(rectangle_a, rectangle_b) == False

In [42]:
def PrintMaximalSet(maximal_set):
    for i, rectangle in enumerate(maximal_set):
        if i < len(maximal_set) - 1:
            print("{}, ".format(rectangle.uuid), end = '')
        else: 
            print("{}".format(rectangle.uuid), end = '')

In [43]:
def PrintMaximalSets(maximal_sets):
    for maximal_set in sorted(maximal_sets, key=len, reverse=True):
        PrintMaximalSet(maximal_set)
        print('')

In [44]:
def FindMaximalSetsFromRectangles(rectangles):
    maximal_sets = []
    
    for rectangle in rectangles:
        if rectangle.assigned_to_set: continue

        single_maximal_set = []
        single_maximal_set.append(rectangle)
        rectangle.assigned_to_set = True

        for candidate in rectangles:
            if candidate.uuid == rectangle.uuid: continue
            if CheckIntersectionOfRectangle(rectangle, candidate) \
                and CheckCandidateIntersectsWithSetMembers(candidate, single_maximal_set):
                single_maximal_set.append(candidate)
                candidate.assigned_to_set = True
        maximal_sets.append(single_maximal_set)
    
    return maximal_sets

In [45]:
rectangle_1 = UUIDRectangle(1, 1, 5, 5, 1)
rectangle_2 = UUIDRectangle(4, 4, 3, 3, 2)
rectangle_3 = UUIDRectangle(6, 1, 1, 1, 3)
rectangle_4 = UUIDRectangle(2, 2, 3, 3, 4)
rectangle_5 = UUIDRectangle(1, 4, 1, 3, 5)

rectangles = [rectangle_1, rectangle_2, rectangle_3, rectangle_4, rectangle_5]

maximal_sets = FindMaximalSetsFromRectangles(rectangles)
            
PrintMaximalSets(maximal_sets)

1, 2, 4
5, 1
3


In [46]:
def CreateRectanglesFromData():
    rectangles = []
    data = np.loadtxt(os.getcwd() + '/rectangles.txt')

    for i in range(0, data.shape[0]):
        rectangle = UUIDRectangle(data[i,0], data[i,0], data[i,0], data[i,0], i)
        rectangles.append(rectangle)
    
    return rectangles

In [50]:
def CreateSetOfRectangles(rectangles):
    rectangle_set = set()
    
    for i, r in enumerate(rectangles):
        rectangle_set.add(r)
        
    return rectangle_set

In [51]:
rectangle_set = CreateSetOfRectangles(rectangles)

In [52]:
def GetIntersectionMembers(curr, rectangles):
    remainder = rectangles

    # x axis
    remainder.sort(key=lambda r: r.x + r.width)
    list_bc2 = [r.x + r.width for r in remainder] 
    idx = bisect_right(list_bc2, curr.x)
    remainder = remainder[max(0,idx):]

    remainder.sort(key=lambda r: r.x)
    list_bc1 = [r.x for r in remainder] 
    idx = bisect_left(list_bc1, curr.x + curr.width)
    remainder = remainder[0:min(idx, len(remainder))]

    # y axis
    remainder.sort(key=lambda r: r.y + r.height)
    list_bc2 = [r.y + r.height for r in remainder] 
    idx = bisect_right(list_bc2, curr.y)
    remainder = remainder[max(0,idx):]

    remainder.sort(key=lambda r: r.y)
    list_bc1 = [r.y for r in remainder] 
    idx = bisect_left(list_bc1, curr.y + curr.height)
    remainder = remainder[0:min(idx, len(remainder))]
    
    intersection_members = remainder
    
    return intersection_members
    

In [112]:
def FindMaximalSetsFromRectanglesRecursion(max_sets, max_sets_uuids, rectangle, rectangles):
    intersection_members = GetIntersectionMembers(rectangle, rectangles)
    is_max_set = True

    for i, candidate in enumerate(intersection_members):
        if candidate.uuid == rectangle.uuid: continue

        candidate_intersection_members = GetIntersectionMembers(candidate, intersection_members)
        if set(candidate_intersection_members) != set(intersection_members):
            is_max_set = False
            FindMaximalSetsFromRectanglesRecursion(max_sets, max_sets_uuids, candidate,
                    list(set(candidate_intersection_members) - set(intersection_members)) \
                    + list(set(intersection_members) - set(candidate_intersection_members)))
        else: # max set
            max_set = True
            break

    if is_max_set:
        max_sets.append(intersection_members)
        uuids = [m.uuid for m in intersection_members]
        max_sets_uuids.update(uuids)
    
    return max_sets, max_sets_uuids

In [113]:
def OuterLoop(rectangles):
    max_sets = []
    uuids = set()

    for rectangle in rectangles:
        if rectangle.uuid in uuids: continue
        max_sets, uuids = FindMaximalSetsFromRectanglesRecursion(max_sets, uuids, rectangle, rectangles)

    return max_sets

In [114]:
rectangle_1 = UUIDRectangle(1, 1, 5, 5, 1)
rectangle_2 = UUIDRectangle(4, 4, 3, 3, 2)
rectangle_3 = UUIDRectangle(6, 1, 1, 1, 3)
rectangle_4 = UUIDRectangle(2, 2, 3, 3, 4)
rectangle_5 = UUIDRectangle(1, 4, 1, 3, 5)

rectangles = [rectangle_1, rectangle_2, rectangle_3, rectangle_4, rectangle_5] 

PrintMaximalSets(OuterLoop(rectangles))

4, 1, 2
3




In [56]:
maximal_sets = []
uuids = set()
rectangles = CreateRectanglesFromData()[0:50]
PrintMaximalSets(FindMaximalSetsFromRectanglesRecursion(maximal_sets, uuids, rectangles))

42, 43, 3, 8, 16, 23, 38, 1, 39, 13, 37, 24, 17, 27, 31, 32, 46, 12, 25, 28, 0, 5, 14, 18, 21, 22, 45
19, 36, 41, 7, 47, 49
33, 34, 48




In [25]:
len(rectangles)

100